## Efficient calculations with Dask

Inside cubes, Iris already supports lazy data.
operations in various statistical functions and cube arithmetic.


# basic winds code (NOT FUNCTIONAL)
mean direction, mean + std-dev of windspeedwind_speed = (u*u + v*v) ** 0.5
time_mean_windspd = wind_speed.collapsed('time', iris.analysis.MEAN)
time_stdev_windspd = wind_speed.collapsed('time', iris.analysis.STD_DEV)
wind_dirs = u.copy()
wind_dirs.rename('wind_direction')
wind_dirs.data = da.atan2(u.lazy_data(), v.lazy_data())
wind_dirs.units = 'radians'
time_mean_wind_dirs = wind_dirs.collapsed('time', iris.analysis.MEAN)

In [ ]:
# basic winds code (NOT FUNCTIONAL)

In [1]:
import iris
from iris import sample_data_path
import glob
import dask
import dask.array as da


In [2]:
filepaths = glob.glob(sample_data_path('UM', '*.pp'))
filepath = filepaths[0]
print filepath

/home/h05/itpp/git/iris-sample-data/iris_sample_data/sample_data/UM/northward_sea_ice_velocity.1890.01.01.00.00.pp


In [ ]:
cube = iris.load_cube(filepath)
print cube

In [ ]:
print cube.has_lazy_data()

In [ ]:
print type(cube.core_data())

In [ ]:
# Define an wrapper to show us when the actual data is fetched.

class Arraylike_AccessFlagged(object):
    def __init__(self, array, name='A'):
        self.dtype = array.dtype
        self.shape = array.shape
        self._array = array
        self._name = name

    def __getitem__(self, keys):
        print "\nArray fetch {}[{}]".format(self._name, keys)
        return self._array[keys]

def showaccess_lazy(data, chunks=None):
    if chunks is None:
        chunks = data.shape
    return da.from_array(Arraylike_AccessFlagged(data), chunks)

In [ ]:
import numpy as np
real_test_data = np.arange(8.).reshape((2, 4))
test_lazy_data = showaccess_lazy(real_test_data, chunks=(1, 4))
print test_lazy_data

In [ ]:
# This just shows us when data fetches occur.
test_lazy_data.compute()

In [ ]:
# Now make a cube, derive a statistical calculation, and show what happens when we compute results.
test_cube = cube.copy(data=showaccess_lazy(cube.data, chunks=(1, 360)))


In [ ]:
test_cube

In [ ]:
print test_cube

In [ ]:
test_cube.copy().data[:2, :4]

In [ ]:
test_zonal_average = test_cube.collapsed(['longitude'], iris.analysis.MEAN)

In [ ]:
test_zonal_average[:3].copy().data

In [ ]:
test_zonal_stdev = test_cube.collapsed(['longitude'], iris.analysis.STD_DEV)

In [ ]:
test_zonal_stdev[:2].copy().data

In [ ]:
test_cube.has_lazy_data()

In [ ]:
test_cube = iris.load_cube(filepaths)
test_zonal_average = test_cube.collapsed(['longitude'], iris.analysis.MEAN)
test_zonal_stdev = test_cube.collapsed(['longitude'], iris.analysis.STD_DEV)

In [ ]:
%%timeit
av = test_zonal_average.copy().data

In [ ]:
%%timeit
stdev = test_zonal_stdev.copy().data

In [ ]:
%%timeit
av, stdev = da.compute(test_zonal_average.lazy_data(), test_zonal_stdev.lazy_data())

In [ ]:
# BECAUSE
test_cube = iris.load_cube(filepaths)
test_cube = test_cube.copy(data=showaccess_lazy(test_cube.lazy_data(), chunks=test_cube.shape))
test_zonal_average = test_cube.collapsed(['longitude'], iris.analysis.MEAN)
test_zonal_stdev = test_cube.collapsed(['longitude'], iris.analysis.STD_DEV)

In [ ]:
%%timeit -n 1 -r 1
av = test_zonal_average.copy().data
print av[100, 20:23]

In [ ]:
%%timeit -n 1 -r 1
stdev = test_zonal_stdev.copy().data
print stdev[100, 20:23]

In [ ]:
%%timeit -n 1 -r 1
av, stdev = da.compute(test_zonal_average.lazy_data(), test_zonal_stdev.lazy_data())
print av[100, 20:23]
print stdev[100, 20:23]

In [ ]:
len(filepaths)

In [3]:
u, v = iris.load_cube(filepaths[:60]), iris.load_cube(filepaths[60:])
for cube in (u, v):
    for name in ('forecast_reference_time', 'forecast_period'):
        cube.remove_coord(name)

In [4]:
# Alternative:
u = iris.load_cube(filepaths)
v = u.copy()

In [5]:
        
u.coord('time').bounds = None
v.coord('time').bounds = None
v.coord('time').points = u.coord('time').points
v.rename('eastward_sea_ice_velocity')
#print u
#print
#print v
cube_windspeed = (u*u + v*v) ** 0.5
cube_windspeed.rename('wind_speed')

print cube_windspeed
print
print 'cube_windspeed.has_lazy_data() = ', cube_windspeed.has_lazy_data()

wind_speed / (m.s-1)                (time: 120; latitude: 215; longitude: 360)
     Dimension coordinates:
          time                           x              -               -
          latitude                       -              x               -
          longitude                      -              -               x
     Auxiliary coordinates:
          forecast_period                x              -               -
     Scalar coordinates:
          forecast_reference_time: 1859-09-01 00:00:00

cube_windspeed.has_lazy_data() =  True


In [ ]:
spds = cube_windspeed.copy().data
print {name: getattr(np, name)(spds) for name in ('min', 'max', 'mean', 'std')}

In [6]:
cube_winddir = cube_windspeed.copy()
cube_winddir.rename('wind_direction')
cube_winddir.unit = 'degrees'
cube_winddir.data = da.rad2deg(da.arctan2(u.lazy_data(), v.lazy_data()))

print cube_winddir
print
print 'cube_winddir.has_lazy_data() = ', cube_winddir.has_lazy_data()

wind_direction / (m.s-1)            (time: 120; latitude: 215; longitude: 360)
     Dimension coordinates:
          time                           x              -               -
          latitude                       -              x               -
          longitude                      -              -               x
     Auxiliary coordinates:
          forecast_period                x              -               -
     Scalar coordinates:
          forecast_reference_time: 1859-09-01 00:00:00

cube_winddir.has_lazy_data() =  True


In [ ]:
angs = cube_winddir.copy().data
print {name: getattr(np, name)(angs) for name in ('min', 'max', 'mean', 'std')}

In [7]:
cube_mean_windspeed = cube_windspeed.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)
cube_stdev_windspeed = cube_windspeed.collapsed(('latitude', 'longitude'), iris.analysis.STD_DEV)
cube_mean_ang = cube_winddir.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)
cube_stdev_ang = cube_winddir.collapsed(('latitude', 'longitude'), iris.analysis.STD_DEV)


/home/h05/itpp/git/iris/iris_main/lib/iris/cube.py:3158: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
/home/h05/itpp/git/iris/iris_main/lib/iris/coords.py:1209: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'longitude'.
  warnings.warn(msg.format(self.name()))


In [8]:
%%timeit
cube_mean_windspeed.copy().data

1 loop, best of 3: 441 ms per loop


In [9]:
%%timeit
cube_stdev_windspeed.copy().data

1 loop, best of 3: 521 ms per loop


In [10]:
%%timeit
cube_mean_ang.copy().data

1 loop, best of 3: 384 ms per loop


In [11]:
%%timeit
cube_stdev_ang.copy().data

1 loop, best of 3: 436 ms per loop


In [12]:
%%timeit
da.compute(cube_mean_windspeed.core_data(), cube_stdev_windspeed.core_data(), cube_mean_ang.core_data(), cube_stdev_ang.core_data())

1 loop, best of 3: 1.3 s per loop


In [ ]:
import datetime
def timeop(op):
    t0 = datetime.datetime.now()
    op()
    t1 = datetime.datetime.now()
    return (t1 - t0).total_seconds()

stats_cubes = [cube_mean_windspeed, cube_stdev_windspeed, cube_mean_ang, cube_stdev_ang]
assert all([cube.has_lazy_data() for cube in stats_cubes])
time_total = 0.0
for cube in stats_cubes:
    time = timeop(lambda: cube.copy().data)
    print 'Fetch {} = {:0.2f}'.format(cube.name().rjust(30), time)
    time_total += time

print 'Total individual fetch+calculate times        = {:0.2f}'.format(time_total)
time_combined = timeop(lambda: da.compute(*[cube.lazy_data() for cube in stats_cubes]))
percent_speedup = 100.0*(1.0 - time_combined / time_total)
print 'Combine-all-together fetch+calculate times    = {:0.2f}'.format(time_combined)
print '  --> speedup ~{:02.0f}%'.format(percent_speedup)


In [ ]:

test_zonal_average

In [ ]:
for cube in (cube_mean_windspeed, cube_mean_ang):
    print cube.has_lazy_data()

In [ ]:
cube_mean_windspeed.lazy_data().visualize()